**IOC 19 – Unauthorized Service Creation via Event ID 7045 (Host Triage)**
Category: Host System Activity Indicator
IOC Type: Persistence via service registration
Telemetry Origin: Windows Event Logs (System), specifically Event ID 7045

**1. Attacker Analogy**
The Night Janitor Who Installs His Own Office Keycard Reader

Imagine a janitor who’s supposed to clean up and leave. Instead, he quietly drills a new keycard reader into the wall, one that lets him back in any time — even when security’s off duty. He doesn’t smash a window or pry open a door — he installs his own access mechanism, disguised as part of the building.

That’s what the attacker is doing here. They’re not just running a program once — they’re installing it as a Windows service that runs every time the machine boots, using a name like WinUpdateHelper to blend in. It’s persistence, not just intrusion. They want to stay — indefinitely.

**2. IOC / Source of IOC (Telemetry Origin)**
IOC Artifact:
A Windows Event Log entry: Event ID 7045, indicating a new service was installed.

Telemetry Source:

Windows Event Viewer → System Log

Source: Service Control Manager

Event ID: 7045

Raw Simulated Event Log:


Log Name:      System
Source:        Service Control Manager
Date:          2024-05-21 13:45:02
Event ID:      7045
Level:         Information
Computer:      TARGET-HOST01
Description:
A service was installed in the system.

Service Name:  WinUpdateHelper
Service File Name:  C:\\Windows\\Temp\\updatehelper.exe
Service Type:  user mode service
Service Start Type: auto start
Service Account: LocalSystem
Soft Interpretation:
The system didn’t crash. No antivirus alert went off. But a service was installed that will now launch on every reboot. The name mimics a Windows update tool, but the file path (C:\Windows\Temp) is a red flag — real system services don’t live in Temp. This is quiet persistence — not chaos, but infiltration.

**3. Triage Framework Declaration**
Triage Type: Host-Based Indicator
Canonical Toolset (in required sequence):

**Windows Event Logs** – Confirm Event ID 7045 entries for unrecognized services. We are reviewing the System log in Windows Event Viewer and filtering for Event ID 7045, which records all service installation attempts. However, not all 7045 entries are malicious — system updates, legitimate software, and IT agents can also create services. So we’re looking specifically for unrecognized service names, unexpected file paths (e.g., Temp, AppData), and unusual timing (e.g., services created late at night or shortly before an incident).

What to ask yourself: Does the service name look like standard Windows? Does the path belong to a trusted vendor? Was this installed during a known maintenance window? You’re not proving guilt — you’re identifying anomalies worth chasing.



**EDR Telemetry** – (If available) Correlate with process tree for parent-child relationship. What we’re doing:
Using EDR (Endpoint Detection and Response) tools such as CrowdStrike, SentinelOne, or Microsoft Defender for Endpoint, we pivot to the host that generated the 7045 log. The EDR platform should reveal the endpoint identity (e.g., TARGET-LAPTOP03) and let us explore the process tree that led to the service install. We’re particularly interested in the parent-child relationship — e.g., did the service get created by cmd.exe, powershell.exe, or some unknown binary?

EDR can help us determine:

Which host was affected

Which user account ran the command

Which executable initiated the install

Whether there were prior steps (e.g., phishing doc → macro → install)

Clarification: When the soft interpretation says “EDR shows you who installed it,” that refers to both the user identity and the executing process. And yes — that does mean we can usually confirm which specific laptop or server it came from, and trace it internally through that host's OS stack.

**File System and Registry Inspection** – Examine executable location and associated service keys. What we’re doing:
On the affected endpoint, we investigate the location where the new service binary was installed. In this case, C:\Windows\Temp\updatehelper.exe is already a red flag. We use tools like Autoruns, sc query, Get-WmiObject, or direct regedit queries to locate:

The binary file itself: Confirm size, signature, hash (using tools like Get-FileHash or third-party scanners)

The registry entry: Under HKLM\SYSTEM\CurrentControlSet\Services\WinUpdateHelper we will find configuration data, including:

ImagePath – where the service points to

Start – startup type (0 = boot, 2 = auto, 3 = manual)

Type – whether it's a user-mode or kernel-mode service

Goal: Confirm that the service is not just suspicious in name and path, but also improperly registered, unsigned, or linked to known malware behavior.

Volatile Memory Capture – Verify in-memory presence of the service binary and runtime behavior

Soft Interpretation:
The log (7045) is your motion detector — something was installed. The EDR shows you who installed it. The file system reveals where it lives, and memory capture tells you what it’s doing right now. That’s the order of triage: detect → attribute → locate → analyze.

**4. Host OS Layer with Key Behaviors**
OS Layer	Behavior Observed
Layer 2 – Startup/Persistence	Auto-start service created with suspicious name/path
Layer 3 – Background Services	Service runs as LocalSystem, gaining elevated privileges



Soft Interpretation:
The attacker didn’t drop malware in plain sight. They stitched themselves into the operating system’s boot sequence. It’s like placing a sleeper agent in the morning shift — there’s no “execution event” you’ll catch later. It’s already baked in.

Quick Summary Table: OS Layer Mapping to Metadata
OS Layer	Function	Triggering Metadata	Field
Layer 2	Startup / Persistence	Service Start Type: auto start	Confirms auto-launch on boot
Service Name: WinUpdateHelper	Suspicious disguise as system utility
Layer 3	Background Services / System Thread	Service File Name: C:\Windows\Temp\updatehelper.exe	Non-standard executable path
Service Type: user mode service	Confirms silent background service
Service Account: LocalSystem	Indicates high-privilege execution


**5. Cross-Layer Interaction Pivots**
Layer 2 → Layer 3: Service registration (persistence) initiates background execution.

Layer 3 → Layer 6 (Network): If the binary initiates outbound command-and-control (C2) or data exfiltration

Soft Interpretation:
This isn’t just a service registration — it may be a staging platform. Once installed, it could begin calling home, opening reverse shells, or monitoring the system. One event can unlock multi-layer control.

**6. OSI Layer Relevance**
OSI Layer 7 – Application: Windows Service Manager interface.

OSI Layer 6 – Presentation (Optional): If encryption is used in outbound C2

OSI Layer 3 – Network: If the service reaches out via IP to an external actor

Soft Interpretation:
This case starts at the application layer — the creation of a service — but may push downward into network-layer visibility if the binary opens external communications. You may not need Wireshark yet, but it could be your next move.

**7. Attacker Behavior Interpretation**
This is a stealth persistence move, not an initial intrusion. The attacker has already gained access and is now embedding themselves for long-term control. The use of a system-like name (WinUpdateHelper), autostart configuration, and the choice of LocalSystem privileges indicate a goal of invisibility plus power. There’s no user prompt, no GUI, no malware splash — just a quiet new process in the background.

**8. Defender Action Summary**
Search for Event ID 7045 entries on high-value systems

Review all service entries with suspicious names or unexpected file paths (especially under Temp, AppData, or non-standard folders)

Validate service binaries via hash or static analysis

Review service account privileges — if running as LocalSystem, risk is elevated

Remove unauthorized services and monitor for reinstallation attempts

Hunt laterally — this host may be a pivot point

Soft Interpretation:
The event isn’t the end — it’s the start of a deeper investigation. One 7045 entry is a signpost. The real question is: Who installed it? When? Is it alone?

**9. Attacker Strategy Notes**
The attacker is operating post-compromise, seeking long-term access. They’re embedding persistence via Windows services because:

It’s native to the OS (no external tools required)

It’s stealthy (doesn’t raise alarms unless monitored)

It runs under elevated privileges
This behavior suggests either initial staging for broader compromise or survivability across reboots. Their confidence is in the invisibility — not the sophistication.

**10. Who’s Who – Object Role Clarification**
Object	Role in the Case
WinUpdateHelper	Name of the unauthorized service registered by attacker
C:\Windows\Temp\updatehelper.exe	Suspicious binary path used to disguise service as a system update
Event ID 7045	Windows Event Log ID that records service installation events
LocalSystem	High-privilege service account attacker used to gain system-level access
Service Control Manager	Windows subsystem that logs creation and status of services

**11. Softened Interpretations (Per Section)**
Section 1: The attacker isn’t crashing the system — they’re registering themselves as part of it.

Section 2: The telemetry doesn’t scream “breach.” But one quiet entry says, “A new system-level service has arrived.”

Section 4: This isn’t payload execution — it’s persistence insertion.

Section 5: One line of service creation creates a domino path from boot to background to beacon.

Section 7: They don’t need to hide the malware — they just need to make it look like it belongs.

**Conclusion / Analyst README**
This case study captures a classic example of attacker persistence using a built-in operating system feature: the Windows service framework. Through the lens of Event ID 7045, the attacker installed a background service named WinUpdateHelper, masked to resemble a legitimate update utility. The binary resides in a non-standard path (C:\Windows\Temp), auto-starts at boot, and runs with LocalSystem privileges — all red flags.

The observable indicator comes from host telemetry, specifically the Windows System log. From there, the investigation unfolds with precise triage: checking the service metadata, correlating process origins through EDR, inspecting the executable on disk, and validating its presence in memory.

The OS layer mapping spans both:

Layer 2 – Startup/Persistence, due to auto-launch configuration

Layer 3 – Background Services, based on privileged, silent execution

Though no network behavior was seen in this phase, the potential for Layer 6/Layer 3 engagement remains — this service could act as a staging mechanism for future command-and-control or lateral movement.

From a CySA+ readiness perspective, this case reinforces essential skills:

Interpreting raw Windows Event Log telemetry

Connecting service-related IOCs to host OS behavior

Using structured triage across tools: event logs, EDR, file inspection, memory analysis

Mapping host artifacts to OSI layers with clarity

This IOC is not about noise — it’s about quiet control. It shows how a single, routine-looking log entry can point to an entire compromise. The defender’s job is to recognize the subtlety, follow the telemetry, and stop the system from running code it never should have trusted.





